<H3>Load libraries</H3>

In [1]:
import numpy as np
import os
import random
import matplotlib.pyplot as plt
import pandas as pd
import gc
from sklearn.cross_validation import train_test_split

%matplotlib inline

<H3>Load train and test data</H3>

In [2]:
train = pd.read_csv('training_dataset.csv')
test = pd.read_csv('test_dataset.csv')

In [3]:
print train.shape
print test.shape

(486048, 54)
(207424, 48)


<H3>Count the number of emails per user and number of opens by an user</H3>
<H4>Find also the ratio between those two variables</H4>

In [4]:
train['user_count'] = train.groupby(['user_id'])['user_id'].transform('count')
train['open_count'] = train.groupby(['user_id'])['opened'].transform('sum')
train['open_ratio'] = train['open_count'] / train['user_count'] 
user_info = train[['user_id', 'user_count', 'open_count', 'open_ratio']]

In [5]:
user_info = user_info.drop_duplicates()

In [6]:
test = pd.merge(test, user_info, on = 'user_id', how = 'left')

In [7]:
import time
import datetime

<H3>Convert time formats</H3>

In [8]:
lo_mean = train.last_online.mean()
train.ix[train['last_online'].isnull(), 'last_online'] = lo_mean
test.ix[test['last_online'].isnull(), 'last_online'] = lo_mean

In [9]:
train['last_online_t'] = train.apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x['last_online'] )), axis = 1)
train['hacker_created_t'] = train.apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x['hacker_created_at'] )), axis = 1)
train['sent_time_t'] = train.apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x['sent_time'] )), axis = 1)

In [10]:
train['last_online_t'] = pd.to_datetime(train['last_online_t'])
train['hacker_created_t'] = pd.to_datetime(train['hacker_created_t'])
train['sent_time_t'] = pd.to_datetime(train['sent_time_t'])

In [11]:
train['sent_date'] = train.apply(lambda x: x['sent_time_t'].date().day, axis = 1)

<H3>Create dummy variables</H3>

In [12]:
sent_date_dum = pd.get_dummies(train['sent_date'], prefix = 'sd')
train = train.join(sent_date_dum)

In [13]:
train['sent_day'] = train.apply(lambda x: x['sent_time_t'].strftime('%a'), axis = 1)

In [14]:
sent_day_dum = pd.get_dummies(train['sent_day'], prefix = 'sda')
train = train.join(sent_day_dum)

In [15]:
train['sent_hr'] = train.apply(lambda x: x['sent_time_t'].time().hour, axis = 1)

In [16]:
sent_hr_dum = pd.get_dummies(train['sent_hr'], prefix = 'sh')
train = train.join(sent_hr_dum)

In [17]:
train['hack_yr'] = train.apply(lambda x: (2017 - x['hacker_created_t'].date().year), axis = 1)

In [18]:
hack_yr_dum = pd.get_dummies(train['hack_yr'], prefix = 'hy')
train = train.join(hack_yr_dum)

In [19]:
train['hr_gap'] = train.apply(lambda x: (x['sent_time_t'] - x['last_online_t']).days, axis = 1)

In [20]:
train.ix[train['hacker_timezone'].isnull(), 'hacker_timezone'] = 1.800000e+04
tz_dum = pd.get_dummies(train['hacker_timezone'], prefix = 'tz')
train = train.join(tz_dum)

In [21]:
test['last_online_t'] = test.apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x['last_online'] )), axis = 1)
test['hacker_created_t'] = test.apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x['hacker_created_at'] )), axis = 1)
test['sent_time_t'] = test.apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x['sent_time'] )), axis = 1)

In [22]:
test['last_online_t'] = pd.to_datetime(test['last_online_t'])
test['hacker_created_t'] = pd.to_datetime(test['hacker_created_t'])
test['sent_time_t'] = pd.to_datetime(test['sent_time_t'])

In [23]:
test['sent_date'] = test.apply(lambda x: x['sent_time_t'].date().day, axis = 1)

In [24]:
sent_date_dum = pd.get_dummies(test['sent_date'], prefix = 'sd')
test = test.join(sent_date_dum)

In [25]:
test['sent_day'] = test.apply(lambda x: x['sent_time_t'].strftime('%a'), axis = 1)

In [26]:
sent_day_dum = pd.get_dummies(test['sent_day'], prefix = 'sda')
test = test.join(sent_day_dum)

In [27]:
test['sent_hr'] = test.apply(lambda x: x['sent_time_t'].time().hour, axis = 1)

In [28]:
sent_hr_dum = pd.get_dummies(test['sent_hr'], prefix = 'sh')
test = test.join(sent_hr_dum)

In [29]:
test['hack_yr'] = test.apply(lambda x: (2017 - x['hacker_created_t'].date().year), axis = 1)

In [30]:
hack_yr_dum = pd.get_dummies(test['hack_yr'], prefix = 'hy')
test = test.join(hack_yr_dum)

In [31]:
test['hr_gap'] = test.apply(lambda x: (x['sent_time_t'] - x['last_online_t']).days, axis = 1)

In [32]:
test.ix[test['hacker_timezone'].isnull(), 'hacker_timezone'] = 1.800000e+04
tz_dum = pd.get_dummies(test['hacker_timezone'], prefix = 'tz')
test = test.join(tz_dum)

In [33]:
mail_cat_dum = pd.get_dummies(train['mail_category'],prefix='mc')
train = train.join(mail_cat_dum)
mail_cat_dum_test = pd.get_dummies(test['mail_category'],prefix='mc')
test = test.join(mail_cat_dum_test)

In [34]:
train_columns = list(train.columns)
test_columns = list(test.columns)

In [35]:
[i for i in train_columns if i not in test_columns]

['open_time',
 'click_time',
 'unsubscribe_time',
 'clicked',
 'opened',
 'unsubscribed',
 'mc_mail_category_13',
 'mc_mail_category_15',
 'mc_mail_category_17',
 'mc_mail_category_18']

In [36]:
train_cols_del = ['user_id', 'mail_id', 'mail_type', 'open_time', 'click_time', 'unsubscribe_time', 'unsubscribed', 'clicked', 
                  'contest_login_count_1_days', 'ipn_count_1_days', 'ipn_read_1_days', 'mail_type', 'mail_category', 
                 'hacker_timezone', 'last_online', 'hacker_created_at', 'sent_time', 'last_online_t',
                 'hacker_created_t', 'sent_time_t', 'sent_date', 'sent_day', 'sent_hr', 'hack_yr', 'hr_gap', 
                  'mc_mail_category_13', 'mc_mail_category_15', 'mc_mail_category_17', 'mc_mail_category_18']
test_cols_del = ['user_id', 'mail_id','mail_type', 'contest_login_count_1_days', 'ipn_count_1_days', 'ipn_read_1_days', 
                 'mail_type', 'mail_category', 'hacker_timezone', 'last_online', 'hacker_created_at', 
                 'sent_time', 'last_online_t', 'hacker_created_t', 'sent_time_t', 'sent_date', 'sent_day', 'sent_hr', 'hack_yr',
                'hr_gap']

In [37]:
train.drop(train_cols_del, axis=1, inplace=True)
test.drop(test_cols_del, axis=1, inplace=True)

In [38]:
print train.shape
print test.shape

(486048, 145)
(207424, 144)


<H3>Save cleaned data as csv files</H3>

In [39]:
train.to_csv('train_clean.csv', index = False)
test.to_csv('test_clean.csv', index = False)